In [1]:
import time
import torch
import torch.nn.functional as F


bz = 32
seq_len = 2048
dims = 64
n_heads = 8
q = torch.randn(bz, n_heads, seq_len, dims, dtype=torch.float16).cuda()
k = torch.randn(bz, n_heads, seq_len, dims, dtype=torch.float16).cuda()
v = torch.randn(bz, n_heads, seq_len, dims, dtype=torch.float16).cuda()

dropout_rate = 0.2
num_trials = 10

with torch.backends.cuda.sdp_kernel(
    enable_flash=True, enable_math=False, enable_mem_efficient=False
):
    start = time.time()
    for i in range(num_trials):
        out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_rate)
    end = time.time()
    print('Flash attention took {} seconds'.format(end - start))

# with torch.backends.cuda.sdp_kernel(
#     enable_flash=False, enable_math=True, enable_mem_efficient=False
# ):
#     start = time.time()
#     for i in range(num_trials):
#         out = F.scaled_dot_product_attention(q, k, v, dropout_p=dropout_rate)
#     end = time.time()
#     print('scled dot product attention disabled flash took {} seconds'.format(end - start))

start = time.time()
for i in range(num_trials):
    attn = q @ k.transpose(-2, -1)
    attn = attn.softmax(dim=-1)
    attn = F.dropout(attn, p=dropout_rate, training=True)
    x = (attn @ v).transpose(1, 2)  # .reshape(bz, seq_len, n_heads*dims)
end = time.time()
print('Standard attention took {} seconds'.format(end - start))

Flash attention took 0.0031659603118896484 seconds
Standard attention took 0.405972957611084 seconds


: 